In [1]:
import numpy as np
from qiskit import execute, QuantumCircuit
from qiskit.test.mock import FakeParis
from qiskit.providers.aer import QasmSimulator, noise

from qiskit.ignis.verification.tomography import marginal_counts, expectation_counts
from qiskit.ignis.mitigation.measurement import (
    FullMeasureErrorMitigationGenerator,
    FullMeasureErrorMitigator,
    fit_full_measure_error_mitigator,
    TensorMeasureErrorMitigationGenerator,
    TensorMeasureErrorMitigator,
    fit_tensor_measure_error_mitigator,
    counts_expectation_value)

## Sim noise model

In [2]:
backend = FakeParis()
coupling_map = backend.configuration().coupling_map
sim = QasmSimulator()

# Example max qubit number
num_qubits = 4

# Gate Error
nm_noro = noise.NoiseModel.from_backend(backend, readout_error=False)

# Gate and Readout Error
nm = noise.NoiseModel.from_backend(backend, readout_error=False)
for i in range(num_qubits):
    p_error1 = (i + 1) * 0.01
    p_error0 = 2 * p_error1
    nm.add_readout_error([[1 - p_error0, p_error0], [p_error1, 1 - p_error1]], [i])

# Readout Error only
nm_ro = noise.NoiseModel()
for i in range(num_qubits):
    p_error1 = i * 0.01
    p_error0 = 2 * p_error1
    nm_ro.add_readout_error([[1 - p_error0, p_error0], [p_error1, 1 - p_error1]], [i])

seed_simulator = 1101

Device reported a gate error parameter greater than maximum allowed value (1.000000 > 0.800000). Truncating to maximum value.
Device model returned a depolarizing error parameter greater than maximum allowed value (1.068139 > 1.066667). Truncating to maximum value.
Device reported a gate error parameter greater than maximum allowed value (1.000000 > 0.800000). Truncating to maximum value.
Device model returned a depolarizing error parameter greater than maximum allowed value (1.068276 > 1.066667). Truncating to maximum value.
Device reported a gate error parameter greater than maximum allowed value (1.000000 > 0.800000). Truncating to maximum value.
Device model returned a depolarizing error parameter greater than maximum allowed value (1.068139 > 1.066667). Truncating to maximum value.
Device reported a gate error parameter greater than maximum allowed value (1.000000 > 0.800000). Truncating to maximum value.
Device model returned a depolarizing error parameter greater than maximum al

### Cal (Full)

Ideal cal with only readout error

In [3]:
circuits1, metadata1, _ = FullMeasureErrorMitigationGenerator(num_qubits).run()

job_cal1 = execute(circuits1, sim,
                   seed_simulator=seed_simulator,
                   shots=100000,
                   basis_gates=nm_ro.basis_gates,
                   noise_model=nm_ro,
                   backend_options={'method': 'density_matrix'})
result_cal1 = job_cal1.result()

In [4]:
full_mitigator = fit_full_measure_error_mitigator(result_cal1, metadata1)

### Cal (Tensor)

In [5]:
circuits2, metadata2, _ = TensorMeasureErrorMitigationGenerator(num_qubits).run()

job_cal2 = execute(circuits2, sim,
                   seed_simulator=seed_simulator,
                   shots=100000,
                   basis_gates=nm_ro.basis_gates,
                   noise_model=nm_ro,
                   backend_options={'method': 'density_matrix'})
result_cal2 = job_cal2.result()

In [6]:
tensor_mitigator = fit_tensor_measure_error_mitigator(result_cal2, metadata2)

### Expval (all-qubits)

In [7]:
# Test Circuit
qc = QuantumCircuit(4, 4)
qc.z(0)
qc.x(1)
qc.h(2)
qc.cx(2, 3)
qc.measure([0, 1, 2, 3], [0, 1, 2, 3])

# Simulation
shots = 5000
seed = 1101
result_target = execute(qc, sim, shots=shots, seed_simulator=seed,
                        coupling_map=coupling_map,
                        basis_gates=nm_noro.basis_gates,
                        noise_model=nm_noro).result()
result_noise = execute(qc, sim, shots=shots, seed_simulator=seed,
                       coupling_map=coupling_map,
                       basis_gates=nm_noro.basis_gates,
                       noise_model=nm).result()
counts_target = result_target.get_counts(0)
counts_noise = result_noise.get_counts(0)

# Expvals
expval_target = counts_expectation_value(counts_target)
expval_nomit = counts_expectation_value(counts_noise)
expval_ainv_full = full_mitigator.expectation_value(counts_noise)
expval_ainv_tp = tensor_mitigator.expectation_value(counts_noise)

expval_fits = np.array([expval_nomit, expval_ainv_tp, expval_ainv_full])
print('Target expval:', expval_target)
print('Fitted Expvals:', expval_fits)
print('Differences:', np.abs(expval_fits - expval_target).round(5))

Target expval: -0.0256
Fitted Expvals: [-0.1532     -0.07081566 -0.07182556]
Differences: [0.1276  0.04522 0.04623]


### Expval (all-qubits, permuted)

In [8]:
for meas_qubits in [[0, 1, 2, 3], [0, 1, 3, 2], [1, 3, 0, 2], [3, 0, 1, 2], [2, 0, 3, 1], [3, 2, 1, 0]]:

    # Test Circuit
    qc = QuantumCircuit(4, 4)
    qc.z(0)
    qc.x(1)
    qc.h(2)
    qc.cx(2, 3)
    qc.measure(meas_qubits, range(4))

    # Simulation
    shots = 10000

    result_target = execute(qc, sim, shots=shots,
                            coupling_map=coupling_map,
                            noise_model=nm_noro,
                            seed_simulator=seed_simulator).result()
    result_noise = execute(qc, sim, shots=shots,
                           coupling_map=coupling_map,
                           noise_model=nm,
                           seed_simulator=seed_simulator).result()
    counts_target = result_target.get_counts(0)
    counts_noise = result_noise.get_counts(0)

    # Expvals
    expval_target = counts_expectation_value(counts_target)
    expval_nomit = counts_expectation_value(counts_noise)

    expval_ainv_tp = tensor_mitigator.expectation_value(counts_noise, qubits=meas_qubits)
    expval_ainv_full = full_mitigator.expectation_value(counts_noise, qubits=meas_qubits)

    print('\nclbit_map:', meas_qubits)
    print('Target expval:', expval_target)
    print('Tensor-mit Expval:', expval_ainv_tp)
    print('Full-mit Expvals:', expval_ainv_full)
    print('Nomit Expval:', expval_nomit)


clbit_map: [0, 1, 2, 3]
Target expval: -0.0288
Tensor-mit Expval: -0.06788708465230699
Full-mit Expvals: -0.06891448479259937
Nomit Expval: -0.1514

clbit_map: [0, 1, 3, 2]
Target expval: -0.0288
Tensor-mit Expval: -0.06781692455348971
Full-mit Expvals: -0.06884773726299474
Nomit Expval: -0.1514

clbit_map: [1, 3, 0, 2]
Target expval: -0.0288
Tensor-mit Expval: -0.07507108366124665
Full-mit Expvals: -0.07607917940984894
Nomit Expval: -0.1576

clbit_map: [3, 0, 1, 2]
Target expval: -0.0288
Tensor-mit Expval: -0.06632128154236064
Full-mit Expvals: -0.06736408115933692
Nomit Expval: -0.1502

clbit_map: [2, 0, 3, 1]
Target expval: -0.0288
Tensor-mit Expval: -0.0632075723453543
Full-mit Expvals: -0.06423500477344485
Nomit Expval: -0.1474

clbit_map: [3, 2, 1, 0]
Target expval: -0.0288
Tensor-mit Expval: -0.06411899952234218
Full-mit Expvals: -0.06514867034815008
Nomit Expval: -0.1476


### Expval Partial Qubits

In [9]:
for qubits in [[0, 1], [0, 2], [0, 3], [3, 1], [2, 1], [3, 2], [2, 3]]:
    
    # Test Circuit
    qc = QuantumCircuit(4, 2)
    qc.h(qubits[0])
    qc.u2(0.2, 0.1, qubits[0])
    qc.cx(qubits[0], qubits[1])
    qc.barrier(qubits)
    qc.cx(qubits[0], qubits[1])
    qc.measure(qubits, [0, 1])

    # Simulation
    shots = 5000
    seed = 1101
    result_target = execute(qc, sim, shots=5000,
                            seed_simulator=seed,
                            coupling_map=coupling_map,
                            basis_gates=nm_noro.basis_gates,
                            noise_model=nm_noro).result()
    result_noise = execute(qc, sim, shots=shots,
                           seed_simulator=seed,
                           coupling_map=coupling_map,
                           basis_gates=nm_noro.basis_gates,
                           noise_model=nm).result()
    counts_target = result_target.get_counts(0)
    counts_noise = result_noise.get_counts(0)

    # Expvals
    expval_target = counts_expectation_value(counts_target)
    expval_nomit = counts_expectation_value(counts_noise)
    expval_ainv_tp = tensor_mitigator.expectation_value(counts_noise, qubits=qubits)
    expval_ainv_full = full_mitigator.expectation_value(counts_noise, qubits=qubits)
    expval_fits = np.array([expval_nomit, expval_ainv_tp, expval_ainv_full])

    print('Target expval:', expval_target)
    print('Fitted Expvals:', expval_fits)
    print('Differences:', np.abs(expval_fits - expval_target).round(5))

Target expval: 0.9848
Fitted Expvals: [0.8964     0.93359663 0.93532807]
Differences: [0.0884  0.0512  0.04947]
Target expval: 0.9884
Fitted Expvals: [0.9012     0.97947988 0.97754543]
Differences: [0.0872  0.00892 0.01085]
Target expval: 0.98
Fitted Expvals: [0.8792     0.99922988 0.99797029]
Differences: [0.1008  0.01923 0.01797]
Target expval: 0.9096
Fitted Expvals: [0.8288     0.8609745  0.86016676]
Differences: [0.0808  0.04863 0.04943]
Target expval: 0.9604
Fitted Expvals: [0.8756     0.90987195 0.90852366]
Differences: [0.0848  0.05053 0.05188]
Target expval: 0.9424
Fitted Expvals: [0.8316     0.89885567 0.89811251]
Differences: [0.1108  0.04354 0.04429]
Target expval: 0.9424
Fitted Expvals: [0.7908     0.89143185 0.88871621]
Differences: [0.1516  0.05097 0.05368]
